# Filtering Ofcom Wireless Telegraphy Register

The Wireless Telegraphy Register (WTR) is a publicly available database containing all of the UK's licencing information. The WTR contains information such as licence number, issue date, coordinates and licence holder among many others. The scope of the WTR is much larger than is required to visualise released spectrum in for Wireless Broad Access and will require extensive filtering and conditional formatting.

The WTR contains every licence, over all of the UK's spectrum. Filters must be developed to isolate mid-band licences used exclusively for WBA and must exclude extraneous information such as antenna specifications and device location.

## WTR Complications

Unlike the RRL, the required data exists in a single CSV file, however, there aren't any Ofcom classifications for WBA licences which means that filter criteria need developed very carefully to ensure all non-WBA licences are excluded.

In [16]:
import pandas as pd
import numpy as np

wtr = pd.read_csv('WTR.csv')

# Remove unnecessary columns
wtr = wtr[['Licence Number', 
    'Frequency (Hz)', 
    'Station Type', 
    'Channel Width (Hz)', 
    'Product Description']]

# Convert Hz to MHz
wtr['Frequency (Hz)'] = (wtr['Frequency (Hz)'] / 1000000).round(5)
wtr['Channel Width (Hz)'] = (wtr['Channel Width (Hz)'] / 1000000).round(5)

# Change column names
wtr.rename(columns={'Frequency (Hz)' : 'FREQUENCY',
    'Channel Width (Hz)' : 'BW',
    'Licence Number' : 'LICENCE_NO',
    'Station Type' : 'STATION_TYPE',
    'Product Description' : 'CATEGORY'}, inplace=True)

# Isolate mid-band frequencies
wtr = wtr[wtr['FREQUENCY'] < 7000]
wtr.sort_values(by=['LICENCE_NO','FREQUENCY'], inplace=True)
wtr.reset_index(drop=True, inplace=True)

# Save dataset to CSV
wtr.to_csv('Ofcom Datasets/spectrumLicences.csv', index=False)
wtr

,LICENCE_NO,FREQUENCY,STATION_TYPE,BW,CATEGORY
0,0000036/2,165.58750,T,0.0125,BR Tech Assigned
1,0000036/2,165.58750,R,0.0125,BR Tech Assigned
2,0000036/2,170.38750,R,0.0125,BR Tech Assigned
3,0000036/2,170.38750,T,0.0125,BR Tech Assigned
4,0000042/1,157.02500,T,0.0250,CSR Marina
...,...,...,...,...,...
159903,ES0089008/1,2106.40625,T,4.3000,Satellite (Non Fixed Satellite Earth Station)
159904,ES0089008/1,2216.00000,R,1.7600,Satellite (Non Fixed Satellite Earth Station)
159905,ES0089008/1,2216.00000,R,2.7300,Satellite (Non Fixed Satellite Earth Station)
159906,ES0089008/1,2232.50000,R,4.1400,Satellite (Non Fixed Satellite Earth Station)


## User Generated Dataframe

There is a user generated dataset containing 100 MHz frequency summaries for mid-band wireless broadband licences below 5 GHz.

> NOTE: Using the transpose functions breaks indexing, ignore strange index name

In [79]:
userGen = pd.read_csv('Ofcom Datasets/userGeneratedDataset.csv')
userGen = userGen.set_index('Frequency').transpose()
userGen.reset_index(inplace=True)

# Rename Columns
userGen = userGen.rename(columns={'index' : 'FREQUENCY',
    'Usage (MHz)' : 'BW', 
    'Uplink Usage (MHz)' : 'UP_BW',
    'Downlink Usage (MHz)' : 'LW_BW',
    'Link Usage (MHz)' : 'L_BW'}, inplace=False)

#userGen['BW'] = userGen['BW'].fillna(0)
userGen['FREE'] = 100 - userGen['BW']

# Separate Uplink, Downlink, Total and Free
uplink = userGen[['FREQUENCY','UP_BW','Uplinks']]
uplink['TYPE'] = 'UPLINK'
uplink.rename(columns={'UP_BW':'BW','Uplinks':'Licences'}, inplace=True)

downlink = userGen[['FREQUENCY', 'LW_BW', 'Downlinks']]
downlink['TYPE'] = 'DOWNLINK'
downlink.rename(columns={'LW_BW':'BW','Downlinks':'Licences'}, inplace=True)

link = userGen[['FREQUENCY', 'L_BW', 'Links']]
link['TYPE'] = 'LINK'
link.rename(columns={'L_BW':'BW','Links':'Licences'}, inplace=True)

#total = userGen[['FREQUENCY', 'BW', 'Complete Licences']]
#total['TYPE'] = 'TOTAL'
#total.rename(columns={'Complete Licences':'Licences'}, inplace=True)

free = userGen[['FREQUENCY', 'FREE']]
free['Licences'] = 1
free['TYPE'] = 'UNALLOCATED'
free.rename(columns={'FREE':'BW'}, inplace=True)

userGenFreq = pd.concat([uplink, downlink, link, free], axis=0)
userGenFreq = userGenFreq.astype({'FREQUENCY' : 'int64'})
userGenFreq = userGenFreq.sort_values(by=['FREQUENCY'], inplace=False)

userGenFreq.to_csv('Ofcom Datasets/userGenFreqList.csv', index=False)
userGenFreq

C:\Users\Nav\AppData\Local\Temp\ipykernel_1256\1481661842.py:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\Nav\AppData\Local\Temp\ipykernel_1256\1481661842.py:18: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\Nav\AppData\Local\Temp\ipykernel_1256\1481661842.py:21: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-

Frequency,FREQUENCY,BW,Licences,TYPE
0,700,30.0,3.0,UPLINK
0,700,0.0,0.0,LINK
0,700,36.0,1.0,UNALLOCATED
0,700,34.0,5.0,DOWNLINK
1,800,49.9,7.0,UPLINK
...,...,...,...,...
42,4900,NaN,NaN,UPLINK
43,5000,NaN,NaN,DOWNLINK
43,5000,NaN,NaN,UPLINK
43,5000,NaN,NaN,LINK


# DATA VISUALISATION

## User Generated Frequency List

In [80]:
import plotly_express as px

fig = px.treemap(userGenFreq, path=[px.Constant('1 - 5GHz'), 'FREQUENCY', 'TYPE'], values='BW')
fig.update_traces(root_color="lightgrey")
fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))
fig.write_html('Ofcom Treemaps/userGenFreqTreemap.html')
fig.show()

C:\Users\Nav\AppData\Local\Programs\Python\Python310\lib\site-packages\plotly\express\_core.py:1637: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\Nav\AppData\Local\Programs\Python\Python310\lib\site-packages\plotly\express\_core.py:1637: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\Nav\AppData\Local\Programs\Python\Python310\lib\site-packages\plotly\express\_core.py:1637: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

